In [1]:
from base import BoWS, OneVsAllGridClassifier
import io

from sklearn.metrics import f1_score,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from glob import glob

from os import path
import os

from collections import Counter


import gc

In [2]:
def read_texts(filename):
    with io.open(filename, newline='\n') as filin:
        return filin.readlines()

def get_array(X, idxs):
    return [ X[idx] for idx in idxs ]
def read_dataset(pathname):
    texts = read_texts(path.join(pathname, 'texts.txt'))
    scores = read_texts(path.join(pathname, 'score.txt'))
    scores = list(map(int, scores))
    return texts,scores
def dump_svmlight_file_gz(X,y,filename):
    with gzip.open(filename, 'w') as filout:
        dump_svmlight_file(X, y, filout, zero_based=False)
def load_splits_ids(folddir):
    splits = []
    with open(folddir, encoding='utf8', errors='ignore') as fileout:
        for line in fileout.readlines():
            train_index, test_index = line.split(';')
            train_index = list(map(int, train_index.split()))
            test_index = list(map(int, test_index.split()))
            splits.append( (train_index, test_index) )
    return splits 

In [22]:
datasetdir = "../../../../Documentos/datasets/only_docs_scores/irony/"
train_test_splits = load_splits_ids(path.join(datasetdir, 'representations', 'split_5.csv' ))
texts,scores = read_dataset(datasetdir)

In [23]:
(train_ids, test_ids) = train_test_splits[0]
X_train = get_array(texts, train_ids)
y_train = get_array(scores, train_ids)

bows = BoWS(min_df=2)
X_train_transformed = bows.fit_transform(X_train, y_train)

weak_clf = LogisticRegression(random_state=42, max_iter=1000)
weak_params = {'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None], 'solver': ['liblinear'], 'C': [1, 10, 0.1, 0.01]}

#weak_clf = LinearSVC(random_state=42)
#weak_params = {'C': [1, 10, 0.1, 0.01]}

meta_clf = DecisionTreeClassifier()
meta_params = { 'criterion': [ "gini", "entropy" ], 'max_depth': [None, 2, 4, 6], 'min_samples_split': [2,4,6], 'min_samples_leaf': [1, 2, 4, 6] }

oal = OneVsAllGridClassifier( weak_params, weak_clf, meta_params, meta_clf )

y_pred = oal.fit_predict(X_train_transformed, y_train)

X_test = get_array(texts, test_ids)
X_test_transformed = bows.transform(X_test)

y_test = get_array(scores, test_ids)
gc.collect()

Building class representations: 100%|██████████| 63/63 [00:00<00:00, 1271.40it/s]
/home/mangaravite/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Building class transformation: 100%|██████████| 18/18 [00:00<00:00, 2130.53it/s]


632

In [20]:
import json

In [21]:
with open('tmp_output/caralho.json', 'w') as fp:
    json.dump({'vai': 'tomar', 'no': 'cu'}, fp)

In [7]:
y_pred_test = oal.predict(X_test_transformed)

In [8]:
confusion_matrix(y_test, y_pred_test, labels=oal.classes_)/len(y_test)

array([[0.33333333, 0.16666667, 0.        ],
       [0.16666667, 0.05555556, 0.        ],
       [0.22222222, 0.        , 0.05555556]])

In [10]:
f1_micr = f1_score(y_test, y_pred_test, average='micro')
f1_macr = f1_score(y_test, y_pred_test, average='macro')
print("F1_mi: %.3f and F1_ma: %.3f" % ( f1_micr, f1_macr) )

F1_mi: 0.444 and F1_ma: 0.376


In [11]:
for c in X_test_transformed:
    y_pred = oal.clf_by_class[c].predict(X_test_transformed[c])
    y_test_transformed = oal.transform_y(y_test, c)
    f1_micr = f1_score(y_pred, y_test_transformed, average='micro')
    f1_macr = f1_score(y_pred, y_test_transformed, average='macro')
    print("%d F1_mi: %.3f and F1_ma: %.3f" % ( c, f1_micr, f1_macr) )

-1 F1_mi: 0.444 and F1_ma: 0.375
0 F1_mi: 0.778 and F1_ma: 0.438
1 F1_mi: 0.722 and F1_ma: 0.419


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [5]:
sorted(glob("../../../../Documentos/datasets/only_docs_scores/*"), key=lambda x: len(read_dataset(x)[1]) )

['../../../../Documentos/datasets/only_docs_scores/irony',
 '../../../../Documentos/datasets/only_docs_scores/sarcasm',
 '../../../../Documentos/datasets/only_docs_scores/aisopos_ntua',
 '../../../../Documentos/datasets/only_docs_scores/nikolaos_ted',
 '../../../../Documentos/datasets/only_docs_scores/sentistrength_bbc',
 '../../../../Documentos/datasets/only_docs_scores/sentistrength_myspace',
 '../../../../Documentos/datasets/only_docs_scores/sentistrength_rw',
 '../../../../Documentos/datasets/only_docs_scores/sentistrength_digg',
 '../../../../Documentos/datasets/only_docs_scores/debate',
 '../../../../Documentos/datasets/only_docs_scores/sentistrength_youtube',
 '../../../../Documentos/datasets/only_docs_scores/sanders',
 '../../../../Documentos/datasets/only_docs_scores/vader_amazon',
 '../../../../Documentos/datasets/only_docs_scores/english_dailabor',
 '../../../../Documentos/datasets/only_docs_scores/vader_twitter',
 '../../../../Documentos/datasets/only_docs_scores/sentistren

In [35]:
for datasetdir in sorted(glob("../../../../Documentos/datasets/only_docs_scores/*"), key=lambda x: len(read_dataset(x)[1]) ):
    texts,scores = read_dataset(datasetdir)
    train_test_splits = load_splits_ids(path.join(datasetdir, 'representations', 'split_5.csv' ))
    dname = path.basename(datasetdir)
    print(dname)
    for f, (train_ids, test_ids) in enumerate(train_test_splits):
        X_train = get_array(texts, train_ids)
        y_train = get_array(scores, train_ids)

        bows = BoWS(min_df=2)
        X_train_transformed = bows.fit_transform(X_train, y_train)

        weak_clf = LogisticRegression(random_state=42)
        weak_params = {'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None], 'solver': ['liblinear'], 'C': [1, 10, 0.1, 0.01]}
        #weak_params = {'penalty': ['l2'], 'class_weight': [None], 'solver': ['liblinear'], 'C': [0.01]}
        meta_clf = DecisionTreeClassifier()
        meta_params = { 'criterion': [ "gini", "entropy" ] }

        oal = OneVsAllGridClassifier( weak_params, weak_clf, meta_params, meta_clf )

        y_pred = oal.fit_predict(X_train_transformed, y_train)

        X_test = get_array(texts, test_ids)
        X_test_transformed = bows.transform(X_test)

        y_test = get_array(scores, test_ids)

        print("Fold %d" % f)
        for c in X_test_transformed:
            y_pred = oal.clf_by_class[c].predict(X_test_transformed[c])
            y_test_transformed = oal.transform_y(y_test, c)
            f1_micr = f1_score(y_pred, y_test_transformed, average='micro')
            f1_macr = f1_score(y_pred, y_test_transformed, average='macro')
            print("\t%d F1_mi: %.3f and F1_ma: %.3f" % ( c, f1_micr, f1_macr) )

        gc.collect()
    

Building class representations: 100%|██████████| 63/63 [00:00<00:00, 2006.80it/s]
/home/mangaravite/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Building Models: 100%|██████████| 3/3 [00:00<00:00, 436.19it/s]

irony



Building class transformation: 100%|██████████| 18/18 [00:00<00:00, 2522.55it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
Building class representations: 100%|██████████| 64/64 [00:00<00:00, 1938.99it/s]
/home/mangaravite/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Building Models: 100%|██████████| 3/3 [00:00<00:00, 455.99it/s]

Fold 0
	-1 F1_mi: 0.611 and F1_ma: 0.579
	0 F1_mi: 0.778 and F1_ma: 0.438
	1 F1_mi: 0.722 and F1_ma: 0.419
irony



Building class transformation: 100%|██████████| 17/17 [00:00<00:00, 931.67it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
Building class representations: 100%|██████████| 65/65 [00:00<00:00, 2108.46it/s]
/home/mangaravite/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Building Models: 100%|██████████| 3/3 [00:00<00:00, 369.97it/s]

...

Fold 2
	-1 F1_mi: 0.803 and F1_ma: 0.803
	1 F1_mi: 0.817 and F1_ma: 0.816
stanford_tweets



/home/mangaravite/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Building class representations: 100%|██████████| 288/288 [00:00<00:00, 2041.84it/s]

Fold 3
	-1 F1_mi: 0.704 and F1_ma: 0.704
	1 F1_mi: 0.648 and F1_ma: 0.643
stanford_tweets



/home/mangaravite/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Building class transformation: 100%|██████████| 288/288 [00:00<00:00, 959.85it/s]


KeyboardInterrupt: 

In [ ]:
for datasetdir in sorted(glob("../../../../Documentos/datasets/only_docs_scores/*")):
    print(datasetdir)
    train_test_splits = load_splits_ids(path.join(datasetdir, 'representations', 'split_5.csv' ))
    texts,scores = read_dataset(datasetdir)
    for f, (train_ids, test_ids) in enumerate(train_test_splits):
        X_train = get_array(texts, train_ids)
        y_train = get_array(scores, train_ids)
        
        print(Counter(y_train))

        bows = BoWS(min_df=2)
        X_train_transformed = bows.fit_transform(X_train, y_train)

        weak_clf = LogisticRegression(random_state=42)
        weak_params = {'penalty': ['l1', 'l2'], 'class_weight': ['balanced', None], 'solver': ['liblinear'], 'C': [1, 10, 0.1, 0.01]}
        #weak_params = {'penalty': ['l2'], 'class_weight': [None], 'solver': ['liblinear'], 'C': [0.01]}
        meta_clf = DecisionTreeClassifier()
        meta_params = { 'criterion': [ "gini", "entropy" ] }

        oal = OneVsAllGridClassifier( weak_params, weak_clf, meta_params, meta_clf )

        y_pred = oal.fit_predict(X_train_transformed, y_train)

        X_test = get_array(texts, test_ids)
        X_test_transformed = bows.transform(X_test)

        y_test = get_array(scores, test_ids)
        gc.collect()


In [ ]:
X_train_transformed